# Label Studio


>[Label Studio](https://labelstud.io/guide/get_started) 是一个开源数据标注平台，为LangChain提供了灵活性，可用于为大型语言模型（LLM）的微调标注数据。它还支持准备自定义训练数据以及通过人类反馈收集和评估响应。

在本指南中，您将学习如何将LangChain管道连接到`Label Studio`，以实现：

- 在单个`Label Studio`项目中汇总所有输入提示、对话和响应。这将所有数据整合在一个地方，便于标注和分析。
- 优化提示和响应，创建用于监督微调（SFT）和基于人类反馈的强化学习（RLHF）场景的数据集。标注的数据可用于进一步训练LLM，提高其性能。
- 通过人类反馈评估模型响应。`Label Studio`提供了一个界面，供人类审查和提供对模型响应的反馈，允许评估和迭代。

## 安装和设置

首先安装最新版本的Label Studio和Label Studio API客户端：

In [ ]:
%pip install --upgrade --quiet langchain label-studio label-studio-sdk langchain-openai langchain-community

接下来，在命令行中运行`label-studio`以在`http://localhost:8080`启动本地LabelStudio实例。有关更多选项，请参见[Label Studio安装指南](https://labelstud.io/guide/install)。

您需要一个令牌来进行API调用。

在浏览器中打开LabelStudio实例，转到`账户和设置 > 访问令牌`并复制密钥。

使用您的LabelStudio URL、API密钥和OpenAI API密钥设置环境变量：

In [ ]:
import os

os.environ["LABEL_STUDIO_URL"] = "<您的LABEL-STUDIO-URL>"  # 例如 http://localhost:8080
os.environ["LABEL_STUDIO_API_KEY"] = "<您的LABEL-STUDIO-API-KEY>"
os.environ["OPENAI_API_KEY"] = "<您的OPENAI-API-KEY>"

## 收集LLM提示和响应

用于标注的数据存储在Label Studio的项目中。每个项目由一个XML配置标识，该配置详细说明了输入和输出数据的规范。

创建一个项目，该项目以文本格式接收人工输入，并在文本区域中输出可编辑的LLM响应：

```xml
<View>
<Style>
    .prompt-box {
        background-color: white;
        border-radius: 10px;
        box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
        padding: 20px;
    }
</Style>
<View className="root">
    <View className="prompt-box">
        <Text name="prompt" value="$prompt"/>
    </View>
    <TextArea name="response" toName="prompt"
              maxSubmissions="1" editable="true"
              required="true"/>
</View>
<Header value="评价响应："/>
<Rating name="rating" toName="prompt"/>
</View>
```

1. 要在Label Studio中创建项目，请点击"创建"按钮。
2. 在"项目名称"字段中输入项目名称，例如`My Project`。
3. 导航至`标注设置 > 自定义模板`并粘贴上面提供的XML配置。

您可以通过`LabelStudioCallbackHandler`连接，在LabelStudio项目中收集输入LLM提示和输出响应：

In [1]:
from langchain_community.callbacks.labelstudio_callback import (
    LabelStudioCallbackHandler,
)

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(
    temperature=0, callbacks=[LabelStudioCallbackHandler(project_name="My Project")]
)
print(llm.invoke("讲个笑话"))

在Label Studio中，打开`My Project`。您将看到提示、响应和模型名称等元数据。

## 收集聊天模型对话

您还可以在LabelStudio中跟踪和显示完整的聊天对话，并能够评价和修改最后的响应：

1. 打开Label Studio并点击"创建"按钮。
2. 在"项目名称"字段中输入项目名称，例如`New Project with Chat`。
3. 导航至标注设置 > 自定义模板并粘贴以下XML配置：

```xml
<View>
<View className="root">
     <Paragraphs name="dialogue"
               value="$prompt"
               layout="dialogue"
               textKey="content"
               nameKey="role"
               granularity="sentence"/>
  <Header value="最终响应："/>
    <TextArea name="response" toName="dialogue"
              maxSubmissions="1" editable="true"
              required="true"/>
</View>
<Header value="评价响应："/>
<Rating name="rating" toName="dialogue"/>
</View>
```

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat_llm = ChatOpenAI(
    callbacks=[
        LabelStudioCallbackHandler(
            mode="chat",
            project_name="New Project with Chat",
        )
    ]
)
llm_results = chat_llm.invoke(
    [
        SystemMessage(content="始终使用大量表情符号"),
        HumanMessage(content="讲个笑话"),
    ]
)

在Label Studio中，打开"New Project with Chat"。点击创建的任务以查看对话历史记录并编辑/注释响应。

## 自定义标注配置

您可以修改LabelStudio中的默认标注配置，添加更多目标标签，如响应情感、相关性和许多[其他类型的标注者反馈](https://labelstud.io/tags/)。

可以从UI添加新的标注配置：转到`设置 > 标注界面`，并使用额外的标签设置自定义配置，如用于情感的`Choices`或用于相关性的`Rating`。请记住，任何配置中都应该包含[`TextArea`标签](https://labelstud.io/tags/textarea)，以显示LLM响应。

或者，您可以在项目创建前的初始调用中指定标注配置：

In [ ]:
ls = LabelStudioCallbackHandler(
    project_config="""
<View>
<Text name="prompt" value="$prompt"/>
<TextArea name="response" toName="prompt"/>
<TextArea name="user_feedback" toName="prompt"/>
<Rating name="rating" toName="prompt"/>
<Choices name="sentiment" toName="prompt">
    <Choice value="积极"/>
    <Choice value="消极"/>
</Choices>
</View>
"""
)

请注意，如果项目不存在，将使用指定的标注配置创建项目。

## 其他参数

`LabelStudioCallbackHandler`接受几个可选参数：

- **api_key** - Label Studio API密钥。覆盖环境变量`LABEL_STUDIO_API_KEY`。
- **url** - Label Studio URL。覆盖`LABEL_STUDIO_URL`，默认为`http://localhost:8080`。
- **project_id** - 现有Label Studio项目ID。覆盖`LABEL_STUDIO_PROJECT_ID`。在此项目中存储数据。
- **project_name** - 如果未指定项目ID，则为项目名称。创建一个新项目。默认为带有当前日期格式的`"LangChain-%Y-%m-%d"`。
- **project_config** - [自定义标注配置](#自定义标注配置)
- **mode**: 使用此快捷方式从头开始创建目标配置：
   - `"prompt"` - 单个提示，单个响应。默认。
   - `"chat"` - 多轮聊天模式。

